# SPOT Kriging in 2 Dimensions: Nyström Approximation vs. Exact (Rosenbrock Function)

This notebook demonstrates how to use the `Spot` class from `spotpython` with and without the Nyström approximation for Kriging surrogates on the 2-dimensional Rosenbrock function.

We use a maximum of 100 function evaluations.

In [ ]:
import numpy as np
from spotpython.fun.objectivefunctions import Analytical
from spotpython.spot import Spot
from spotpython.utils.init import fun_control_init, design_control_init, surrogate_control_init

## Define the  Rosenbrock Function

In [ ]:
# do not change, these are given by the organizers
dim = 6
lower = np.full(dim, -2)
upper = np.full(dim, 2)
fun = Analytical().fun_rosenbrock
fun_evals = 100

## Set up SPOT Controls

## Aquisition Random and Regression and y as Infill Criterion

In [ ]:
init_size = 12
use_nystrom = False
method = "regression"
infill_criterion = "ei"
tolerance_x = 1e-6 # minimum distance in x for accepting new points
seed = 321
max_surrogate_points = 24
min_Lambda = -6
max_Lambda = 1
min_theta = -3
max_theta = 2
isotropic = False
optim_p = True
min_success_rate = 0.10 #  percent
acquisition_failure_strategy = "random"

In [ ]:
fun_control = fun_control_init(
    lower=lower,
    upper=upper,
    fun_evals=fun_evals,
    max_time=60,
    seed=seed,
    show_progress=True,
    infill_criterion=infill_criterion,
    min_success_rate=min_success_rate,
    tolerance_x=tolerance_x,
    TENSORBOARD_CLEAN=True,
    tensorboard_log=True,
    acquisition_failure_strategy=acquisition_failure_strategy,
)
design_control = design_control_init(init_size=init_size)
surrogate_control_exact = surrogate_control_init(use_nystrom=use_nystrom, 
                                                 optim_p=optim_p,
                                                 method=method, max_surrogate_points=max_surrogate_points, min_Lambda=min_Lambda, max_Lambda=max_Lambda, min_theta=min_theta, max_theta=max_theta, isotropic=isotropic)
spot_exact_random = Spot(
        fun=fun,
        fun_control=fun_control,
        design_control=design_control,
        surrogate_control=surrogate_control_exact
    )
spot_exact_random.run()
spot_exact_random.plot_progress(log_y=True, title="spotpython: Random Kriging")

## Aquisition MM and Regression and y as Infill Criterion

In [ ]:
acquisition_failure_strategy = "mm"

In [ ]:
fun_control = fun_control_init(
    lower=lower,
    upper=upper,
    fun_evals=fun_evals,
    max_time=60,
    seed=seed,
    show_progress=True,
    infill_criterion=infill_criterion,
    min_success_rate=min_success_rate,
    tolerance_x=tolerance_x,
    TENSORBOARD_CLEAN=True,
    tensorboard_log=True,
    acquisition_failure_strategy=acquisition_failure_strategy,
)
design_control = design_control_init(init_size=init_size)
surrogate_control_exact = surrogate_control_init(use_nystrom=use_nystrom, optim_p=optim_p, method=method, max_surrogate_points=max_surrogate_points, min_Lambda=min_Lambda, max_Lambda=max_Lambda, min_theta=min_theta, max_theta=max_theta, isotropic=isotropic)
spot_exact_mm = Spot(
    fun=fun,
    fun_control=fun_control,
    design_control=design_control,
    surrogate_control=surrogate_control_exact
)
spot_exact_mm.run()
spot_exact_mm.plot_progress(log_y=True, title="spotpython: Morris-Mitchell Kriging.")

## Compare Minimum Found

In [ ]:
print(f"[2D] spotpython Kriging Random:\n min y = {spot_exact_random.min_y:.4f} at x = {spot_exact_random.min_X}")
print(f"[2D] spotpython Kriging Morris-Mitchell:\n min y = {spot_exact_mm.min_y:.4f} at x = {spot_exact_mm.min_X}")

In [ ]:
spot_exact_random.plot_important_hyperparameter_contour(max_imp=3)


In [ ]:
spot_exact_mm.plot_important_hyperparameter_contour(max_imp=3)

In [1]:
import numpy as np
from spotpython.spot import Spot
from spotpython.fun import Analytical
from spotpython.utils.init import fun_control_init
nn = 3
fun_sphere = Analytical().fun_sphere
fun_control = fun_control_init(
        lower = np.array([-1, -1]),
        upper = np.array([1, 1]),
        n_points=nn,
        )
S = Spot(
    fun=fun_sphere,
    fun_control=fun_control,
    )
S.X = S.design.scipy_lhd(
    S.design_control["init_size"], lower=S.lower, upper=S.upper
)
print(f"S.X: {S.X}")
S.y = S.fun(S.X)
print(f"S.y: {S.y}")
S.fit_surrogate()
x = np.array([0.5, 0.5])
y_infill = S.infill(x)
print(f"y_infill: {y_infill}")
assert np.isscalar(y_infill) or y_infill.shape == (1,)

Seed set to 123
Seed set to 123


S.X: [[ 0.86352963  0.7892358 ]
 [-0.24407197 -0.83687436]
 [ 0.36481882  0.8375811 ]
 [ 0.415331    0.54468512]
 [-0.56395091 -0.77797854]
 [-0.90259409 -0.04899292]
 [-0.16484832  0.35724741]
 [ 0.05170659  0.07401196]
 [-0.78548145 -0.44638164]
 [ 0.64017497 -0.30363301]]
S.y: [1.36857656 0.75992983 0.83463487 0.46918172 0.92329124 0.8170764
 0.15480068 0.00815134 0.81623768 0.502017  ]
y_infill: [0.50052583]


In [6]:
import numpy as np
from spotpython.spot import Spot
from spotpython.fun import Analytical
from spotpython.utils.init import fun_control_init
nn = 3
fun_sphere = Analytical().fun_sphere
fun_control = fun_control_init(
        lower = np.array([-1, -1]),
        upper = np.array([1, 1]),
        n_points=nn,
        )
S = Spot(
    fun=fun_sphere,
    fun_control=fun_control,
    )
z00 = np.array([[1, 2, 3, 4], [5, 6, 7, 8]])
S.var_type = ["float", "int", "int", "float"]
S.process_z00(z00)

Seed set to 123


[np.float64(3.0), np.int64(2), np.int64(3), np.float64(6.0)]